In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key  
gmaps.configure(api_key=g_key)

# Store Part 1 results into DataFrame
    
    * Load the csv exported in part 1 to a DataFrame

In [20]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Saint Paul Harbor,90,US,1592314738,87,57.79,-152.41,50.00,13.87
1,Port Hedland,0,AU,1592314738,68,-20.32,118.57,68.00,6.93
2,Lorengau,63,PG,1592314738,75,-2.02,147.27,82.63,7.85
3,Castro,89,BR,1592314738,82,-24.79,-50.01,61.45,7.87
4,Praia,20,CV,1592314738,66,14.92,-23.51,86.00,5.82


# Humidity Heatmap
Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [21]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [22]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=maxhumidity,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria
Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [23]:
narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Puerto Suárez,0,BO,1592314740,69,-18.95,-57.80,77.00,11.41
1,Cape Town,0,ZA,1592314494,33,-33.93,18.42,71.60,5.82
2,Hermanus,0,ZA,1592314742,49,-34.42,19.23,71.01,1.99
3,Saldanha,0,ZA,1592314745,46,-33.01,17.94,71.60,3.36
4,Martapura,0,ID,1592314749,90,-3.42,114.85,75.07,2.66


# Hotel Map
Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [24]:
hotel_df =  pd.DataFrame(narrowed_city_df, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Puerto Suárez,BO,-18.95,-57.80,
1,Cape Town,ZA,-33.93,18.42,
2,Hermanus,ZA,-34.42,19.23,
3,Saldanha,ZA,-33.01,17.94,
4,Martapura,ID,-3.42,114.85,
5,Katsuura,JP,35.13,140.30,
6,Yumen,CN,40.28,97.20,
7,Jurm,AF,36.86,70.83,
8,Vilhena,BR,-12.74,-60.15,
9,Frutal,BR,-20.02,-48.94,


In [25]:
hotel_name = []

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [26]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()

,City,Country,Lat,Lng,Hotel Name
0,Puerto Suárez,BO,-18.95,-57.80,Hotel casa real
1,Cape Town,ZA,-33.93,18.42,Atlanticview Cape Town Boutique Hotel
2,Hermanus,ZA,-34.42,19.23,The Marine Hermanus
3,Saldanha,ZA,-33.01,17.94,Blue Bay Lodge
4,Martapura,ID,-3.42,114.85,favehotel Banjarbaru - Banjarmasin
5,Katsuura,JP,35.13,140.30,鵜原館
8,Vilhena,BR,-12.74,-60.15,Hotel Mirage
9,Frutal,BR,-20.02,-48.94,Premium Hotel
10,Mikhaylovka,RU,50.06,43.24,Mini-hotel Meduza
11,Erzin,TR,36.96,36.20,Hattusa Vacation Thermal Club Erzin


In [27]:
# # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map


figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Display figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig



Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…